## A tutorial to get you started faster
In this tutorial we will show you a breif example about how our model works, you will learn that quickly
<br/>
We will generate a synthetic network(BA with 256 nodes) and try to compress it to a macro(smaller) network with 30 nodes

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import *
import random
import numpy as np

import torch
import torch_geometric

from model import Encoder,WeightSumDecoder
from data import get_feature,standardize_matrix,get_zt
from utils import mins_cross_entropy,shuffle_rows,visualize_weighted_graph

In [ ]:
torch.__version__,torch_geometric.__version__,np.__version__

In [ ]:
s = 2050
torch.manual_seed(s)
np.random.seed(s)
random.seed(s)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

## get a synthetic network

In [ ]:
g = nx.barabasi_albert_graph(256,2)

In [ ]:
nx.draw(g,node_size=10)

## From the topology, we extract the information for the GNN including:
- Node features
- normalized partition curve(10 points in the curve)
- edge index

In [ ]:
X = get_feature(g).float().to(device)
adj = torch.from_numpy(nx.to_numpy_array(g)).to(device)
y = get_zt(adj).float().to(device)
edge_index = torch_geometric.utils.dense_to_sparse(adj)[0].to(device)

## Define the model and optimizer

In [ ]:
macro_node = 30
encoder = Encoder(input_dim =5, hidden_dim = 128, output_dim = macro_node, n_layers = 1).to(device)
decoder = WeightSumDecoder(macro_node)
op = torch.optim.Adam(encoder.parameters(),lr=0.0001)

## Start Training!

In [ ]:
# record the loss
lss_train = []
lss_ce = []

In [ ]:
for e in tqdm(range(5000)):
    # Partition function evaluation
    group = encoder(X,edge_index)
    new_p = decoder(group,adj.float())
    yhat = get_zt(new_p)
    l0 = torch.mean(torch.abs(y-yhat))
    
    # Cross Entropy Loss exists to make nodes as seperate as possible, it will be helpful for training in most cases
    sgroup = shuffle_rows(group)
    l1 = mins_cross_entropy(sgroup,group)
    
    loss= l0+l1*0.001 # this weight should not be too large, you can try by yourself
    
    # backpropagation and gradient decent
    op.zero_grad()
    loss.backward()
    op.step()
    
    # recording
    lss_train.append(l0.item())
    lss_ce.append(l1.item())

## Visualize the loss curve

In [ ]:
plt.subplot(121)
plt.title('Partition function Difference')
plt.plot(lss_train)
plt.subplot(122)
plt.title('Cross Entropy')
plt.plot(lss_ce)
plt.show()

## Visualize the Macro Network

In [ ]:
visualize_weighted_graph(new_p.cpu().detach().numpy(),np.sum(group.cpu().detach().numpy()*20,axis=0))

## And compare the partition function

In [ ]:
plt.figure(figsize=(5,3))
plt.title('Normalized Partition Function Comparision')
plt.plot(yhat.cpu().detach().numpy(),label='Macro Network')
plt.plot(y.cpu().detach().numpy(),label='Original Network')
plt.xlabel(r'$\tau$',fontsize=15)
plt.ylabel(r'$Z(\tau)$',fontsize=15)
plt.xscale('log')
plt.legend()
plt.show()